In [75]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image as im, ImageDraw, ImageFont
import PIL
from scipy import ndimage
from matplotlib.pyplot import figure
from glob import glob
from utils import add_margin, expand2square, put_in_a_box, binarize
from tqdm import tqdm

from typing import *

In [76]:
PATH_TO_FONT_CHARS = "../formatted_data/chars_images/chars"

PATH_TO_AUGMENTED_CHARS = "../formatted_data/chars_images/augmented_chars"
PATH_TO_ROTATED_CHARS = f"{PATH_TO_AUGMENTED_CHARS}/rotated"
PATH_TO_OFFSET_CHARS = f"{PATH_TO_AUGMENTED_CHARS}/offset"
PATH_TO_CONNECTED_CHARS = f"{PATH_TO_AUGMENTED_CHARS}/connected"
PATH_TO_RESIZED_CHARS = f"{PATH_TO_AUGMENTED_CHARS}/resized"

LOWERCASE = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
UPPERCASE = LOWERCASE.upper().replace("Ъ", '')

In [77]:
class Char:
    def __init__(self, path: str):
        self.path = path
        self.image = im.open(path)
        
        split = os.path.normpath(path).split(os.path.sep)
        self.name = split[-1].replace(".png", "")
        self.font = split[-2]


In [78]:
fonts = os.listdir(PATH_TO_FONT_CHARS)
chars_dict = {font: glob(f"{PATH_TO_FONT_CHARS}/{font}/*") for font in fonts}
for k, v in chars_dict.items():
    chars = []
    for char in v:
        chars.append(Char(char))
    chars_dict[k] = chars

In [79]:
def convert_png_transparent(image, bg_color=(255,255,255)):
    array = np.array(image, dtype=np.ubyte)
    mask = (array[:,:,:3] == bg_color).all(axis=2)
    alpha = np.where(mask, 0, 255)
    array[:,:,-1] = alpha
    return im.fromarray(np.ubyte(array))

def get_dominant_color(img):
    palette_size = 2
    # Resize image to speed up processing
    #img = convert_png_transparent(pil_img.copy())
    img.thumbnail((100, 100))

    # Reduce colors (uses k-means internally)
    paletted = img.convert('P', palette=im.ADAPTIVE, colors=palette_size)

    # Find the color that occurs most often
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=False)
    palette_index = color_counts[0][1]
    dominant_color = palette[palette_index*3:palette_index*3+3]
    dominant_color.append(255)

    return tuple(dominant_color)

def expand(img, top=0, right=0, bottom=0, left=0):
    return add_margin(
             expand2square(img),
             top=top,
             bottom=bottom,
             left=left,
             right=right
           )

def rotate(degree, image):
    return image.rotate(degree, expand=True, fillcolor=(255,255,255), resample=PIL.Image.Resampling.BICUBIC)

def closest_to(char_image, coord: Tuple[int, int]):
    bin_img = binarize(char_image)
    h, w = char_image.size
    distances = list()
    for r_idx in range(w):
        for c_idx in range(h):
            if bin_img[r_idx, c_idx] > 0.5:
                distance = euclideanDistance(coord, (r_idx, c_idx))
                distances.append((r_idx, c_idx, distance))
    return min(distances, key=lambda entry: entry[2])[:2]

def resize(image: "Image", what: str, how_much: float):
    w,h = image.width, image.height
    if what == 'width':  return image.resize((round(w * how_much), h))
    if what == 'height': return image.resize((w, round(h * how_much)))
    raise AssertionError(f"`what` must be either \"width\" or \"height\", not {what}")
    
def euclideanDistance(coordinate1, coordinate2):
    return pow(pow(coordinate1[0] - coordinate2[0], 2) + pow(coordinate1[1] - coordinate2[1], 2), .5)

In [80]:
""" ROTATION FROM -40° to 40° """
for font, charset in chars_dict.items():
    font_specific_path = f"{PATH_TO_ROTATED_CHARS}/{font}"
    os.makedirs(font_specific_path, exist_ok=True)
    for char in charset:
        for degree in range(-40, 41, 5):
            rotate(degree, char.image) \
                .save(f"{font_specific_path}/{char.name}_{degree}.png")


In [81]:
""" MOVING AROUND """
for font, charset in chars_dict.items():
    font_specific_path = f"{PATH_TO_OFFSET_CHARS}/{font}"
    os.makedirs(font_specific_path, exist_ok=True)
    for char in charset:
        for margin, side in [(m, side) for m in range(5,16,5) for side in "lrtb"]:
            if side == "l":
                image = expand(char.image, left=margin)
            elif side == "r":
                image = expand(char.image, right=margin)
            elif side == "t":
                image = expand(char.image, top=margin)
            elif side == "b":
                image = expand(char.image, bottom=margin)
            image.save(f"{font_specific_path}/{char.name}_{side}{margin}.png")

In [82]:
# goes up and right from here
right_bottom_spot = lambda image: (closest_to(image, (image.height, image.width)), {"x": 100, "y": -30})
# goes down and right from here
right_top_spot = lambda image: (closest_to(image, (0, image.height)), {"x": 100, "y": 30})

# goes up and rigth from here
left_bottom_spot = lambda image: (closest_to(image, (image.width, 0)), {"x": 100, "y": -30})
# goes down and left from here
left_top_spot = lambda image: (closest_to(image, (0,0)), {"x": -100, "y": 30})

In [83]:
""" CONNECTED CHARS """
for font, charset in chars_dict.items():
    font_specific_path = f"{PATH_TO_CONNECTED_CHARS}/{font}"
    os.makedirs(font_specific_path, exist_ok=True)
    for char in charset:
        dominant = get_dominant_color(char.image)
        # left_top from prev
        char_copy = char.image.copy()
        draw = ImageDraw.Draw(char_copy) 
        (y, x), destination = left_top_spot(char_copy)
        draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=2, joint="curve")     
        char_copy.save(f"{font_specific_path}/{char.name}_lt.png")
        
        # right_top to next
        char_copy = char.image.copy()
        draw = ImageDraw.Draw(char_copy)
        (y, x), destination = right_top_spot(char_copy)
        draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=2, joint="curve")
        char_copy.save(f"{font_specific_path}/{char.name}_rt.png")
        
        # right_bottom to next
        char_copy = char.image.copy()
        draw = ImageDraw.Draw(char_copy) 
        (y, x), destination = right_bottom_spot(char_copy)
        draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=2, joint="curve")
        char_copy.save(f"{font_specific_path}/{char.name}_rb.png")
        
        # left_bottom to next
        char_copy = char.image.copy()
        draw = ImageDraw.Draw(char_copy) 
        (y, x), destination = left_bottom_spot(char_copy)
        draw.line((x,y, x+destination["x"],y+destination["y"]), fill=dominant, width=2, joint="curve")
        char_copy.save(f"{font_specific_path}/{char.name}_lb.png")

/tmp/ipykernel_325676/576121146.py:15: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  paletted = img.convert('P', palette=im.ADAPTIVE, colors=palette_size)


In [84]:
""" RESIZED, i.e. squashed or extended"""
for font, charset in chars_dict.items():
    font_specific_path = f"{PATH_TO_RESIZED_CHARS}/{font}"
    os.makedirs(font_specific_path, exist_ok=True)
    for char in charset:
        w, h = char.image.size
        resize(char.image, "width",  1.2).save(f"{font_specific_path}/{char.name}_20%wider.png")
        resize(char.image, "width",  1.4).save(f"{font_specific_path}/{char.name}_40%wider.png")
        resize(char.image, "width",  1.6).save(f"{font_specific_path}/{char.name}_60%wider.png")
        resize(char.image, "width",  1.8).save(f"{font_specific_path}/{char.name}_80%wider.png")
        
        resize(char.image, "width",   .8).save(f"{font_specific_path}/{char.name}_20%narrower.png")
        resize(char.image, "width",   .6).save(f"{font_specific_path}/{char.name}_40%narrower.png")
        resize(char.image, "width",   .4).save(f"{font_specific_path}/{char.name}_60%narrower.png")
        
        resize(char.image, "height", 1.2).save(f"{font_specific_path}/{char.name}_20%higher.png")
        resize(char.image, "height", 1.4).save(f"{font_specific_path}/{char.name}_40%higher.png")
        resize(char.image, "height", 1.6).save(f"{font_specific_path}/{char.name}_60%higher.png")
        resize(char.image, "height", 1.8).save(f"{font_specific_path}/{char.name}_80%higher.png")
        
        resize(char.image, "height",  .8).save(f"{font_specific_path}/{char.name}_20%shorter.png")
        resize(char.image, "height",  .6).save(f"{font_specific_path}/{char.name}_40%shorter.png")
        resize(char.image, "height",  .4).save(f"{font_specific_path}/{char.name}_60%shorter.png")
        resize(char.image, "height",  .3).save(f"{font_specific_path}/{char.name}_70%shorter.png")

Создаём фейковые изображения символов в связке с соседними символами и называем это умным словом "аугментация"

In [85]:
def write(font, word):
    img  = im.new("RGBA", (500,150),(255,255,255))
    ImageDraw.Draw(img).text((22, 22), word,(49, 76, 175), font=font)
    return put_in_a_box(img)

def width_of(font, word):
    return write(font, word).width

ttfs = glob("ttfs/*.ttf")

In [86]:
""" SIMULATING A CONTEXT FOR A CHAR """
def simulate(target_char_pool=LOWERCASE,
             left_neighbor_pool=LOWERCASE,
             right_neighbor_pool=LOWERCASE,
             simulations_per_char=33,
             description='simulation'):
    for ttf in ttfs:
        font = ImageFont.truetype(ttf,64)
        fontname = os.path.basename(ttf).replace(".ttf", '')
        font_specific_path = f"../formatted_data/chars_images/augmented_chars/{description}/{fontname}"
        os.makedirs(font_specific_path, exist_ok=True)
        print(fontname)
        for s in tqdm(target_char_pool):
            for idx in range(1,simulations_per_char):
                random = np.random.randint(-50, 50)
                
                f = left_neighbor_pool[random % idx % len(left_neighbor_pool)]
                t = right_neighbor_pool[-random % idx % len(right_neighbor_pool)]
                word = f"{f}{s}{t}"

                img = write(font, word)

                l = img.width - width_of(font, f"{s+t}")
                r = width_of(font, f"{f+s}")
                s_itself = width_of(font, s)
                
                # widen the borders so that the second letter certainly fits in
                if s_itself - 2 <= r - l <= s_itself + 2:
                    l, r = l-5, r+5
                img = img.crop((l, 0, r, img.height))
                
                ## Applying mutations to widen the variety of images
                # 40% chance to rotate
                if 15 <= abs(random) <= 35:
                    img = rotate(random // 2, img)
                # 30% chance to add a shear
                if abs(random) <= 15:
                    p = random / 100
                    img = img.transform((50, 50),
                                        im.Transform.AFFINE,
                                        (1-p*2, 0+p*2, 1, 0, 1, 1),
                                        resample=im.Resampling.BILINEAR)
                # 50% chance to expand or shrink an image in either of two dimensions
                if random % 2 == 0: 
                    img = resize(image=img,
                                 what="width" if abs(random) <= 25 else "height",
                                 how_much=(120 + random) / 100)
                img.save(f"{font_specific_path}/{word}.png")
simulate()

Abram


100%|███████████████████████████████████████████| 33/33 [00:33<00:00,  1.01s/it]


Propisi


100%|███████████████████████████████████████████| 33/33 [00:30<00:00,  1.09it/s]


Gogol


...

100%|███████████████████████████████████████████| 33/33 [00:30<00:00,  1.09it/s]


Denistina


100%|███████████████████████████████████████████| 33/33 [00:30<00:00,  1.10it/s]


Tibalt


100%|███████████████████████████████████████████| 33/33 [00:30<00:00,  1.09it/s]


In [96]:
for a in tqdm("-:,."):
    print(a)

100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 50382.03it/s]

-
:
,
.


In [100]:
target_char_pool="-:,."
left_neighbor_pool= LOWERCASE
right_neighbor_pool=" "

for ttf in ttfs:
    font = ImageFont.truetype(ttf,64)
    fontname = os.path.basename(ttf).replace(".ttf", '')
    font_specific_path = f"../formatted_data/chars_images/augmented_chars/punctuation_marks/{fontname}"
    os.makedirs(font_specific_path, exist_ok=True)
    print(fontname)
    for s in tqdm(target_char_pool):
        for idx in range(1,100):
            try:
                random = np.random.randint(-50, 50)

                f = left_neighbor_pool[random % idx % len(left_neighbor_pool)]
                t = right_neighbor_pool[-random % idx % len(right_neighbor_pool)]
                word = f"{f}{s}{t}"

                img = write(font, word)

                l = img.width - width_of(font, f"{s+t}")
                r = width_of(font, f"{f+s}")
                s_itself = width_of(font, s)

                # widen the borders so that the second letter certainly fits in
                if s_itself - 2 <= r - l <= s_itself + 2:
                    l, r = l-5, r+5
                img = img.crop((l, 0, r, img.height))

                ## Applying mutations to widen the variety of images
                # 40% chance to rotate
                if 15 <= abs(random) <= 35:
                    img = rotate(random // 2, img)
                # 50% chance to expand or shrink an image in either of two dimensions
                if random % 2 == 0: 
                    img = resize(image=img,
                                 what="width" if abs(random) <= 25 else "height",
                                 how_much=(120 + random) / 100)
                img.save(f"{font_specific_path}/{word}_{idx}.png")
            except Exception as e:
                print(e)

Abram


100%|█████████████████████████████████████████████| 4/4 [00:10<00:00,  2.73s/it]


Propisi


100%|█████████████████████████████████████████████| 4/4 [00:11<00:00,  2.79s/it]


Gogol


...

100%|█████████████████████████████████████████████| 4/4 [00:11<00:00,  2.96s/it]


Denistina


100%|█████████████████████████████████████████████| 4/4 [00:11<00:00,  2.89s/it]


Tibalt


100%|█████████████████████████████████████████████| 4/4 [00:11<00:00,  2.85s/it]


In [139]:
ttfs = glob("ttfs/*.ttf")
""" TRASH  """
def trash():
    for ttf in ttfs:
        font = ImageFont.truetype(ttf,64)
        fontname = os.path.basename(ttf).replace(".ttf", '')
        font_specific_path = f"../formatted_data/chars_images/augmented_chars/trash/{fontname}"
        os.makedirs(font_specific_path, exist_ok=True)
        print(fontname)
        for one in tqdm(LOWERCASE):
            for two in LOWERCASE[::-1]:
                for span in np.arange(0.4, 0.51, 0.05):
                    word = f"{one + two}"
                    img = write(font, word)
                    l, r = img.width * span, img.width * (span+0.2)
                    img.crop((l, 0, r, img.height)).save(f"{font_specific_path}/{one}{two}_{round(span, 2)}.png")
trash()

Abram


100%|███████████████████████████████████████████| 33/33 [00:22<00:00,  1.44it/s]


Propisi


100%|███████████████████████████████████████████| 33/33 [00:23<00:00,  1.43it/s]


Gogol


...

100%|███████████████████████████████████████████| 33/33 [00:23<00:00,  1.43it/s]


Denistina


100%|███████████████████████████████████████████| 33/33 [00:22<00:00,  1.44it/s]


Tibalt


100%|███████████████████████████████████████████| 33/33 [00:22<00:00,  1.47it/s]
